In [2]:
import sys
import math
import ctypes
import pyqbdi
import struct

In [3]:
def insnCB(vm, gpr, fpr, data):
    instAnalysis = vm.getInstAnalysis()
    print("0x{:x}: {}".format(instAnalysis.address, instAnalysis.disassembly))
    return pyqbdi.CONTINUE

In [4]:

# get sin function ptr
if sys.platform == 'darwin':
    libmname = 'libSystem.dylib'
elif sys.platform == 'win32':
    libmname = 'api-ms-win-crt-math-l1-1-0.dll'
else:
    libmname = 'libm.so.6'
libm = ctypes.cdll.LoadLibrary(libmname)
funcPtr = ctypes.cast(libm.sin, ctypes.c_void_p).value

# init VM
vm = pyqbdi.VM()

# create stack
state = vm.getGPRState()
addr = pyqbdi.allocateVirtualStack(state, 0x100000)
assert addr is not None

# instrument library and register memory access
vm.addInstrumentedModuleFromAddr(funcPtr)
vm.recordMemoryAccess(pyqbdi.MEMORY_READ_WRITE)

# add callbacks on instructions
vm.addCodeCB(pyqbdi.PREINST, insnCB, None)

# Cast double arg to long and set FPR
arg = 1.0
carg = struct.pack('<d', arg)
fpr = vm.getFPRState()
fpr.xmm0 = carg

# call sin(1.0)
pyqbdi.simulateCall(state, 0x42424242)
success = vm.run(funcPtr, 0x42424242)

# Retrieve output FPR state
fpr = vm.getFPRState()
# Cast long arg to double
res = struct.unpack('<d', fpr.xmm0[:8])[0]
print("%f (python) vs %f (qbdi)" % (math.sin(arg), res))

# cleanup
pyqbdi.alignedFree(addr)


0x7fd470616ca0: 	push	rbp
0x7fd470616ca1: 	push	rbx
0x7fd470616ca2: 	sub	rsp, 40
0x7fd470616ca6: 	mov	rax, qword ptr fs:[40]
0x7fd470616caf: 	mov	qword ptr [rsp + 24], rax
0x7fd470616cb4: 	xor	eax, eax
0x7fd470616cb6: 	stmxcsr	dword ptr [rsp + 8]
0x7fd470616cbb: 	mov	ebx, dword ptr [rsp + 8]
0x7fd470616cbf: 	mov	eax, ebx
0x7fd470616cc1: 	and	ah, -97
0x7fd470616cc4: 	mov	dword ptr [rsp + 16], eax
0x7fd470616cc8: 	movq	rax, xmm0
0x7fd470616ccd: 	sar	rax, 32
0x7fd470616cd1: 	and	eax, 2147483647
0x7fd470616cd6: 	and	ebx, 24576
0x7fd470616cdc: 	jne	1230
0x7fd470616ce2: 	cmp	eax, 1045430271
0x7fd470616ce7: 	jg	71
0x7fd470616d30: 	xor	ebp, ebp
0x7fd470616d32: 	cmp	eax, 1072390143
0x7fd470616d37: 	jg	323
0x7fd470616e80: 	cmp	eax, 1073965308
0x7fd470616e85: 	jg	253
0x7fd470616e8b: 	movq	xmm2, qword ptr [rip + 347085]
0x7fd470616e93: 	movsd	xmm1, qword ptr [rip + 364693]
0x7fd470616e9b: 	movapd	xmm3, xmm0
0x7fd470616e9f: 	movsd	xmm4, qword ptr [rip + 368265]
0x7fd470616ea7: 	andpd	xmm3, xmm2
0x7